# Snowflake Account Usage

### 이 앱은 여러분의 Snowflake 계정에 있는 account_usage 스키마를 기반으로 작동하도록 개발되었습니다. 

자세한 정보는 아래 Snowflake 공식 문서 페이지를 참조하십시오.  
https://docs.snowflake.com/en/sql-reference/account-usage#account-usage-views

Snowflake SE팀의 Nikhil Kolur & Ashish Patel 개발한 Streamlit 버전을 Snowflake Notebooks로 전환하였습니다.  
https://medium.com/snowflake/monitoring-snowflake-with-streamlit-in-snowflake-sis-b00fa02b0d4b

추가적으로 Quick Start의 내용을 함께 포함하였습니다.  
https://quickstarts.snowflake.com/guide/query-cost-monitoring/index.html?index=..%2F..index#0
https://quickstarts.snowflake.com/guide/getting_started_cost_performance_optimization/index.html?index=..%2F..index#0


_Python 코드가 원할하게 수행되기 위해서는 우측상단의 Packages에서 plotly, nbformat을 추가해야 합니다_

# 패키지 설정 및 측정 날짜 지정

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import plotly.express as px 
import datetime
import altair as alt
import plotly.graph_objects as go 
import numpy as np

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
from snowflake.snowpark.window import Window

session = get_active_session()


In [ ]:
st.title('모니터링 날짜 범위 지정')

max_date = datetime.datetime.now()
min_date = datetime.datetime.now() - datetime.timedelta(days=365)
this_year = max_date.year
jan_1 = datetime.date(this_year, 1, 1)
dec_31 = datetime.date(this_year, 12, 31)

if 'starting' not in st.session_state:
    st.session_state.starting = datetime.datetime.now() - datetime.timedelta(days=30)

if 'ending' not in st.session_state:
    st.session_state.ending = max_date

st.markdown("Enter your desired date range (30 days on initial load):")

#Column for Date Picker Buttons
col1, col2, col3, col4, col5 = st.columns([1,1,1,1,1])

with col1:
    if st.button('7 Days'):
            st.session_state.starting = datetime.datetime.now() - datetime.timedelta(days=7)
            st.session_state.ending = datetime.datetime.now()
with col2:
    if st.button('14 Days'):
            st.session_state.starting = datetime.datetime.now() - datetime.timedelta(days=14)
            st.session_state.ending = datetime.datetime.now()
with col3:
    if st.button('30 Days'):
            st.session_state.starting = datetime.datetime.now() - datetime.timedelta(days=30)
            st.session_state.ending = datetime.datetime.now()
with col4:
    if st.button('60 Days'):
            st.session_state.starting = datetime.datetime.now() - datetime.timedelta(days=60)
            st.session_state.ending = datetime.datetime.now()
with col5:
    if st.button('120 Days'):
            st.session_state.starting = datetime.datetime.now() - datetime.timedelta(days=120)
            st.session_state.ending = datetime.datetime.now()

#Date Input
date_input_filter = st.date_input(
    "",
    (st.session_state.starting,st.session_state.ending),
    min_date,
    max_date,
)

#Start and End Date (s = start, e = end)
s,e = date_input_filter

st.divider()

# Get the current credentials
session = get_active_session()
credits_used_df = session.sql

# 사용량 개요

In [ ]:
--Credits Used
select round(sum(credits_used),0) as total_credits 
  from snowflake.account_usage.metering_history 
 where start_time between '{{s}}' and '{{e}}' ;

In [ ]:
--Total # of Jobs Executed
select count(*) as number_of_jobs 
  from snowflake.account_usage.query_history 
 where start_time between '{{s}}' and '{{e}}' ;

In [ ]:
--Current Storage
select round(avg(storage_bytes + stage_bytes + failsafe_bytes) / power(1024, 4),2) as billable_tb 
  from snowflake.account_usage.storage_usage 
 where USAGE_DATE = current_date() - 1;

In [ ]:
st.title('Account 사용량 개요')
 
pandas_credits_used_df = Credits_SQL.to_pandas()
pandas_num_jobs_df = Num_Jobs_SQL.to_pandas()
pandas_current_storage_df = Current_Storage_SQL.to_pandas()

#Final Value
#modified by JH, 2025-04-26 
credits_used_tile = pandas_credits_used_df.iloc[0].values[0]
num_jobs_tile = pandas_num_jobs_df.iloc[0].values[0]
current_storage_tile = pandas_current_storage_df.iloc[0].values[0]

#Column formatting and metrics of header 3 metrics
col1, col2, col3 = st.columns(3)
col1.metric("Credits Used","{:,}".format(int(credits_used_tile))) 
col2.metric("Total # of Jobs Executed","{:,}".format(int(num_jobs_tile))) 
col3.metric("Current Storage (TB)",current_storage_tile)


In [ ]:
--Credits Billed by Month 
select date_trunc('MONTH', usage_date) as Usage_Month, 
       sum(CREDITS_BILLED) as sum_credits
  from snowflake.account_usage.metering_daily_history 
 WHERE usage_date >= DATEADD('MONTH', -12, DATE_TRUNC('MONTH', CURRENT_TIMESTAMP()))
 group by Usage_Month ;


In [ ]:
st.title('월별 전체 크레딧 사용량 추이')

credits_billed_df = Credits_by_Month_SQL.to_pandas()
#st.write(credits_billed_df)
fig_credits_billed=px.bar(credits_billed_df,x='USAGE_MONTH',y='SUM_CREDITS', orientation='v',title="Credits Billed by Month")
st.plotly_chart(fig_credits_billed, use_container_width=True)

st.info('The above chart is static and not modified by the date range filter', icon="ℹ️")

In [ ]:
--Total # of Jobs Executed by Month 
--Added by JH, 2025-05-03
SELECT DATE_TRUNC('MONTH', START_TIME) AS job_month, 
       COUNT(*) AS number_of_jobs                   
  FROM snowflake.account_usage.query_history       
 WHERE START_TIME >= DATEADD('MONTH', -12, DATE_TRUNC('MONTH', CURRENT_TIMESTAMP()))        
 GROUP BY 1                                            
 ORDER BY 1 ASC                                        
;

In [ ]:
st.title('월별 전체 Job 수행 수 변화 추이')

JobByMonth_df = Job_by_Month_SQL.to_pandas()
fig_JobByMonth=px.bar(JobByMonth_df,x='JOB_MONTH',y='NUMBER_OF_JOBS', orientation='v',title="Jobs by Month")
st.plotly_chart(fig_JobByMonth, use_container_width=True)

st.info('The above chart is static and not modified by the date range filter', icon="ℹ️")

# 가상 웨어하우스 (Virtual Warehouse) 분석

In [ ]:
-- Credits Usages of Warehouse
select warehouse_name,sum(credits_used) as total_credits_used 
  from snowflake.account_usage.warehouse_metering_history 
 where start_time between '{{s}}' and '{{e}}' 
 group by 1 
 order by 2 desc 
 limit 10 ;

In [ ]:
st.title('지정된 기간 웨어하우스별 사용된 크레딧')

#Convert to a pandas df
pandas_credits_used_df = Credit_Usage_WH_SQL.to_pandas()

#Added by JH, 2025-04-26 (for descending order)
pandas_credits_used_df_sorted = pandas_credits_used_df.sort_values(by='TOTAL_CREDITS_USED', ascending=True)

#Chart
fig_credits_used=px.bar(pandas_credits_used_df_sorted,x='TOTAL_CREDITS_USED',y='WAREHOUSE_NAME',orientation='h',title="Credits Used by Warehouse")
fig_credits_used.update_traces(marker_color='green')


In [ ]:
--Jobs by Warehouse Data Setup
select warehouse_name,count(*) as number_of_jobs 
  from snowflake.account_usage.query_history 
 where start_time between '{{s}}' and '{{e}}' 
   and warehouse_name is not null  --added by JH, 2025-04-26
 group by 1 
 order by 2 desc 
 limit 10 ;

In [ ]:
st.title('지정된 기간 웨어하우스별 수행된 작업의 수')

#convert to pandas df
pandas_jobs_by_warehouse_df = Jobs_WH_SQL.to_pandas()

#Added by JH, 2025-04-26 (for descending order)
pandas_jobs_by_warehouse_df_sorted = pandas_jobs_by_warehouse_df.sort_values(by='NUMBER_OF_JOBS', ascending=True)

#chart
fig_jobs_by_warehouse=px.bar(pandas_jobs_by_warehouse_df_sorted,x='NUMBER_OF_JOBS',y='WAREHOUSE_NAME',orientation='h',title="# of Jobs by Warehouse")
fig_jobs_by_warehouse.update_traces(marker_color='purple')

In [ ]:
--Average Execution by Query Type
select query_type, warehouse_size, avg(execution_time) / 1000 as average_execution_time 
  from snowflake.account_usage.query_history 
 where start_time between '{{s}}' and '{{e}}' 
 group by 1, 2 
 order by 3 desc;

In [ ]:
st.title('지정된 기간 쿼리 유형별, 웨어하우스 크기별 평균 실행 시간 (초)')

#convert to pandas df
pandas_execution_by_qtype_df = QueryType_SQL.to_pandas()

#Modified by JH, 2025-04-26 
pandas_execution_by_qtype_df=px.bar(pandas_execution_by_qtype_df,x='AVERAGE_EXECUTION_TIME',y='QUERY_TYPE',color='WAREHOUSE_SIZE',orientation='h',title="Average Execution Time by Query Type and Warehouse Size")
st.plotly_chart(pandas_execution_by_qtype_df, use_container_width=True)

In [ ]:
--Credits Used Overtime
select start_time::date as usage_date, warehouse_name, sum(credits_used) as total_credits_used 
  from snowflake.account_usage.warehouse_metering_history 
 where start_time between '{{s}}' and '{{e}}' 
 group by 1,2 
having total_credits_used > 0.001 --Added by JH, 2025-04=26 
 order by 2,1 ;

In [ ]:
st.title('지정된 기간 웨어하우스별 크레딧 사용양 추세')

#convert to pandas df
pandas_credits_used_overtime_df = Credits_Used_Overtime_SQL.to_pandas()

#chart
fig_credits_used_overtime_df=px.bar(pandas_credits_used_overtime_df,x='USAGE_DATE',y='TOTAL_CREDITS_USED',color='WAREHOUSE_NAME',orientation='v',title="Credits Used Overtime")
st.plotly_chart(fig_credits_used_overtime_df, use_container_width=True)

In [ ]:
--Warehouse Variance Overtime
SELECT WAREHOUSE_NAME, 
       DATE(START_TIME) AS DATE, 
       SUM(CREDITS_USED) AS CREDITS_USED, 
       AVG(SUM(CREDITS_USED)) OVER (PARTITION BY WAREHOUSE_NAME ORDER BY DATE ROWS 7 PRECEDING) 
         AS CREDITS_USED_7_DAY_AVG, 
       (TO_NUMERIC(SUM(CREDITS_USED)/CREDITS_USED_7_DAY_AVG*100,10,2)-100)::STRING || '%' 
         AS VARIANCE_TO_7_DAY_AVERAGE 
  FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY 
 where start_time between '{{s}}' and '{{e}}' 
 GROUP BY DATE, WAREHOUSE_NAME 
 ORDER BY DATE DESC 

In [ ]:
st.title('지정된 기간내의 해당 날짜별 이전 7일간의 평균 웨어하우스 사용량 대비 증감양')

#convert to pandas
pandas_warehouse_variance_df = Warehouse_Variance_SQL.to_pandas()

#chart
fig_warehouse_variance_df=px.bar(pandas_warehouse_variance_df,x="DATE",y="VARIANCE_TO_7_DAY_AVERAGE",color ='WAREHOUSE_NAME',orientation='v',title="Warehouse Variance Greater than 7 day Average")
st.plotly_chart(fig_warehouse_variance_df, use_container_width=True)

In [ ]:
with base_table as (
select warehouse_name, 
       total_elapsed_time,
       case
            when total_elapsed_time < 1 then 'a'
            when total_elapsed_time < 10 then 'b'
            when total_elapsed_time < 60 then 'c'
            when total_elapsed_time < 300 then 'd'
            when total_elapsed_time < 600 then 'e'
            when total_elapsed_time > 600 then 'f'
       end as exe_time
  from SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
 where warehouse_name is not null
   and start_time between '{{s}}' and '{{e}}'
   and warehouse_name not ilike 'compute_service%'   
)
select warehouse_name, 
       sum(iff(exe_time= 'a', 1, 0)) u00_01s,
       sum(iff(exe_time= 'b', 1, 0)) u01_10s,
       sum(iff(exe_time= 'c', 1, 0)) u10_60s,
       sum(iff(exe_time= 'd', 1, 0)) u01_05m,
       sum(iff(exe_time= 'e', 1, 0)) u05_10m,
       sum(iff(exe_time= 'f', 1, 0)) u10m_
  from base_table
 group by warehouse_name;


In [ ]:
st.title('지정된 기간 동안 웨어하우스별 워크로드 수행시간 분포')

ExeTimeInWH_df = Exe_Time_in_WH_SQL.to_pandas()

# --- 데이터 준비 ---
plot_columns = ['U00_01S', 'U01_10S', 'U10_60S', 'U01_05M', 'U05_10M', 'U10M_']

# 레이블 매핑 (melt 후의 컬럼명 'variable', 'value' 포함)
labels_map = {
    'WAREHOUSE_NAME': 'Warehouse Name',
    'variable': 'Execution Time Bucket', # melt 후 범례가 될 컬럼
    'value': 'Number of Queries',      # melt 후 y축 값이 될 컬럼
    'U00_01S': '< 1s',                 # variable 컬럼의 값들에 대한 레이블
    'U01_10S': '1s - 10s',
    'U10_60S': '10s - 60s',
    'U01_05M': '1m - 5m',
    'U05_10M': '5m - 10m',
    'U10M_': '> 10m'
}

# --- 데이터 변환 (Wide to Long) ---
# id_vars: 고정할 컬럼 (x축)
# value_vars: 행으로 변환할 컬럼들 (y축 값들)
# var_name: value_vars의 이름들이 들어갈 새 컬럼명 (범례/색상 구분)
# value_name: value_vars의 값들이 들어갈 새 컬럼명 (y축 값)
df_long = pd.melt(ExeTimeInWH_df,
                  id_vars='WAREHOUSE_NAME',
                  value_vars=plot_columns,
                  var_name='variable',  # labels_map의 'variable' 키와 일치
                  value_name='value')   # labels_map의 'value' 키와 일치

# --- Plotly Express로 스택 바 차트 생성 (변환된 데이터 사용) ---
fig = px.bar(
    df_long,                   # 변환된 long-form 데이터 사용
    x='WAREHOUSE_NAME',        # X축: Warehouse 이름 컬럼
    y='value',                 # Y축: melt된 값 컬럼 ('Number of Queries')
    color='variable',          # 색상 구분: melt된 변수 컬럼 ('Execution Time Bucket')
    title='Query Execution Time Distribution by Warehouse',
    labels=labels_map,         # 축 및 범례 레이블 매핑 적용
                               # labels_map이 'variable'과 'value'를 포함하므로 자동 적용됨
    height=500,
    # category_orders를 사용하여 범례 순서 지정 (선택 사항)
    category_orders={'variable': plot_columns} # 원래 컬럼 순서대로 범례 정렬
)

# --- 차트 레이아웃 커스터마이징 (선택 사항) ---
# labels 인자에서 대부분 처리되었으므로 명시적 설정은 필요 없을 수 있음
# fig.update_layout(
#     xaxis_title='Warehouse Name',
#     yaxis_title='Number of Queries',
#     legend_title_text='Execution Time Bucket',
# )
fig.update_yaxes(tickformat=',') # Y축 값에 쉼표 추가

# --- Streamlit에 차트 표시 ---
st.subheader('Execution Time Distribution in Warehouse')
st.plotly_chart(fig, use_container_width=True)


In [ ]:
--Added by MJ, 2025-04-30
select warehouse_name,
       percentile_cont(0.25) within group(order by query_load_percent) as p25,
       median(query_load_percent) as md,
       percentile_cont(0.75) within group(order by query_load_percent) as p75,
  from snowflake.account_usage.query_history
 where warehouse_name is not null
   and query_load_percent is not null
   and start_time between '{{s}}' and '{{e}}'
   and warehouse_name not ilike 'compute_service%'
 group by 1
 order by 3 desc ; 

In [ ]:
st.title('지정된 기간 웨어하우스별 워크로드 사용량 분포 (25%, 50%, 75%)')

WHWorkloadPCT_df = WH_Workload_PCT_SQL.to_pandas()

# Plotly 시각화를 위한 데이터 준비 (오류 막대 길이 계산)
# error_y: 중앙값(MD)에서 위쪽으로 P75까지의 거리
# error_y_minus: 중앙값(MD)에서 아래쪽으로 P25까지의 거리
WHWorkloadPCT_df['error_y_upper'] = WHWorkloadPCT_df['P75'] - WHWorkloadPCT_df['MD']
WHWorkloadPCT_df['error_y_lower'] = WHWorkloadPCT_df['MD']  - WHWorkloadPCT_df['P25']

fig = px.scatter(
    WHWorkloadPCT_df,
    x='WAREHOUSE_NAME',
    y='MD',
    error_y='error_y_upper',      # y값(MD) 기준 +방향 오차 (P75까지)
    error_y_minus='error_y_lower',# y값(MD) 기준 -방향 오차 (P25까지)
    title='Warehouse Query Load Distribution (Median and P25-P75 Range)',
    labels={                      # 축 및 호버 레이블 설정
        'WAREHOUSE_NAME': 'Warehouse',
        'MD': 'Median Query Load (%)',
        'P25': 'P25 Load (%)', # 호버 정보용
        'P75': 'P75 Load (%)'  # 호버 정보용
    },
    hover_data=['P25', 'MD', 'P75'] # 호버 시 표시될 추가 데이터 지정
)

# --- 차트 레이아웃 커스터마이징 ---
fig.update_layout(
    yaxis_title='Query Load Percent (%)', # Y축 제목 명확화
    xaxis_title='Warehouse Name',
    yaxis_range=[0, max(100, WHWorkloadPCT_df['P75'].max() * 1.1)], # Y축 범위 설정 (0 ~ 최대 P75값보다 조금 크게)
    yaxis_ticksuffix='%' # Y축 값 뒤에 '%' 추가
)

# 오류 막대(Error Bars) 스타일 조정 (선택 사항)
fig.update_traces(
    error_y_thickness=1,   # 오류 막대 선 두께
    error_y_width=5        # 오류 막대 상/하단 너비
    # selector=dict(type='scatter') # 특정 trace만 선택할 경우
)

# --- Streamlit에 차트 표시 ---
st.plotly_chart(fig, use_container_width=True)


In [ ]:
--Added by Chanwoo Park, 2025-05-02

SELECT to_date(start_time) as date
      ,warehouse_name
      ,sum(avg_running) as sum_running
      ,sum(avg_queued_load) as sum_queued
  FROM snowflake.account_usage.warehouse_load_history
 WHERE to_date(start_time) >= dateadd(month, -1, current_timestamp())
 GROUP BY 1,2
 ORDER BY 1,2 ;

In [ ]:
# Added by JH, 2025-05-02
st.title('일별 웨어하우스의 워크로드 추이 (지난 1개월)')

RunQueueWH_df = Run_Queue_WH_SQL.to_pandas()

# --- Plotly Express로 라인 차트 생성 ---
# SUM_RUNNING 추이 그래프
fig_running = px.line(
    RunQueueWH_df,
    x='DATE',
    y='SUM_RUNNING',
    color='WAREHOUSE_NAME',          # 웨어하우스별로 다른 색상 라인
    title='Daily Sum of Average Running Queries by Warehouse',
    labels={                         # 축 및 범례 레이블 설정
        'DATE': 'Date',
        'SUM_RUNNING': 'Sum of Avg Running Queries (Daily)',
        'WAREHOUSE_NAME': 'Warehouse'
    },
    markers=True,                   # 각 데이터 포인트에 마커 표시 (선택 사항)
    hover_data={'WAREHOUSE_NAME': True, 'SUM_RUNNING': ':.1f'} # 호버 데이터 형식 지정
)
fig_running.update_layout(
    xaxis_title='Date',
    yaxis_title='Daily Sum of Avg Running Queries',
    hovermode='x unified' # 같은 날짜의 모든 웨어하우스 정보 함께 표시
)
st.plotly_chart(fig_running, use_container_width=True)

# SUM_QUEUED 추이 그래프
fig_queued = px.line(
    RunQueueWH_df,
    x='DATE',
    y='SUM_QUEUED',
    color='WAREHOUSE_NAME',          # 웨어하우스별로 다른 색상 라인
    title='Daily Sum of Average Queued Queries by Warehouse',
    labels={                         # 축 및 범례 레이블 설정
        'DATE': 'Date',
        'SUM_QUEUED': 'Sum of Avg Queued Queries (Daily)',
        'WAREHOUSE_NAME': 'Warehouse'
    },
    markers=True,                   # 각 데이터 포인트에 마커 표시 (선택 사항)
    hover_data={'WAREHOUSE_NAME': True, 'SUM_QUEUED': ':.1f'} # 호버 데이터 형식 지정
)
fig_queued.update_layout(
    xaxis_title='Date',
    yaxis_title='Daily Sum of Avg Queued Queries',
    hovermode='x unified' # 같은 날짜의 모든 웨어하우스 정보 함께 표시
)
# Queued 값이 0인 경우가 많으므로 Y축 범위를 동적으로 설정하거나, 0 이상으로만 설정
min_queued = RunQueueWH_df['SUM_QUEUED'].min()
max_queued = RunQueueWH_df['SUM_QUEUED'].max()
# 0으로만 구성된 경우 max_queued가 0이 되어 range가 [0,0]이 되는것 방지
fig_queued.update_yaxes(range=[min_queued, max(1, max_queued * 1.1)]) # 최소 1 이상의 범위를 가지도록 함

st.plotly_chart(fig_queued, use_container_width=True)

# 쿼리 분석

In [ ]:
--Top 25 Longest Success Queries
select query_id,query_text,(execution_time / 1000) as exec_time 
  from snowflake.account_usage.query_history 
 where execution_status = 'SUCCESS' 
   and start_time between '{{s}}' and '{{e}}' 
   and query_text != '' --Added by JH, 2025-04-26 
   and exec_time > 10   --Added by JH, 2025-04-26
 order by execution_time desc 
 limit 25;

In [ ]:
st.title('지정된 기간동안 가장 오래 수행된 쿼리 리스트')

#convert to pandas df
pandas_longest_queries_df = Longest_Query_SQL.to_pandas()

#Modifed by JH, 2025-04-26
max_len = 100
col_orig = 'QUERY_TEXT'           # 원본 텍스트 컬럼명
col_short = 'QUERY_TEXT_SHORT'    # 새로 만들 짧은 텍스트 컬럼명

# 지정된 길이로 텍스트 자르기
pandas_longest_queries_df[col_short] = pandas_longest_queries_df[col_orig].str.slice(0, max_len)
# 원본 길이가 max_len보다 긴 경우에만 끝에 '...' 추가
pandas_longest_queries_df.loc[pandas_longest_queries_df[col_orig].str.len() > max_len, col_short] += '...'

fig_longest_queries = px.bar(
    pandas_longest_queries_df,
    x='EXEC_TIME',
    y=col_short,          # Y축에는 새로 만든 짧은 텍스트 컬럼 사용
    orientation='h',
    title="Longest Successful Queries (Top 25)",
    hover_data={          # 마우스 호버 시 표시될 정보 설정
        'EXEC_TIME': ':.2f s', # 실행 시간 (소수점 2자리 초 단위 포맷)
        col_orig: True,        # 원본 전체 QUERY_TEXT 표시
        col_short: False       # Y축 레이블과 동일한 짧은 텍스트는 호버에서 숨김 (선택 사항)
    },
    labels={col_short: "Query Text (Truncated)"} # Y축 이름 변경 (선택 사항)
)

st.write(fig_longest_queries)

#chart
#fig_longest_queries=px.bar(pandas_longest_queries_df,x='EXEC_TIME',y='QUERY_TEXT',orientation='h',title="Longest Successful Queries (Top 25) ")
#st.write(fig_longest_queries)

In [ ]:
--Query Modified by JH, 2025-04-26
--Top 25 Failed Queries 
select query_text,count(*) as number_of_execution  
  from snowflake.account_usage.query_history 
 where execution_status = 'FAIL' 
   and start_time between '{{s}}' and '{{e}}' 
   and query_text != '' --Added by JH, 2025-04-26 
 group by 1
having number_of_execution > 1
 order by 2 desc 
 limit 25;

In [ ]:
st.title('지정된 기간동안 가장 많이 실패한 쿼리 문 리스트')

#convert to pandas df
f_pandas_longest_queries_df = Failed_Query_SQL.to_pandas()

#Modifed by JH, 2025-04-26
max_len = 100
col_orig = 'QUERY_TEXT'           # 원본 텍스트 컬럼명
col_short = 'QUERY_TEXT_SHORT'    # 새로 만들 짧은 텍스트 컬럼명

# 지정된 길이로 텍스트 자르기
f_pandas_longest_queries_df[col_short] = f_pandas_longest_queries_df[col_orig].str.slice(0, max_len)
# 원본 길이가 max_len보다 긴 경우에만 끝에 '...' 추가
f_pandas_longest_queries_df.loc[f_pandas_longest_queries_df[col_orig].str.len() > max_len, col_short] += '...'

fig_f_longest_queries = px.bar(
    f_pandas_longest_queries_df,
    x='NUMBER_OF_EXECUTION',
    y=col_short,          # Y축에는 새로 만든 짧은 텍스트 컬럼 사용
    orientation='h',
    title="Most Failed Queries (Top 25)",
    hover_data={          # 마우스 호버 시 표시될 정보 설정
        'NUMBER_OF_EXECUTION': ':3d s', 
        col_orig: True,        # 원본 전체 QUERY_TEXT 표시
        col_short: False       # Y축 레이블과 동일한 짧은 텍스트는 호버에서 숨김 (선택 사항)
    },
    labels={col_short: "Query Text (Truncated)"} # Y축 이름 변경 (선택 사항)
)
fig_f_longest_queries.update_traces(marker_color='red')

st.write(fig_f_longest_queries)

#chart
#fig_f_longest_queries=px.bar(f_pandas_longest_queries_df,x='EXEC_TIME',y='QUERY_TEXT',orientation='h',title="Longest Failed Queries (Top 25)")
#fig_f_longest_queries.update_traces(marker_color='red')
#st.write(fig_f_longest_queries)

In [ ]:
--Total Execution Time by Repeated Queries
select query_text, 
       (sum(execution_time) / 1000) as exec_time,
       count(*) as number_of_query
  from snowflake.account_usage.query_history 
 where execution_status = 'SUCCESS' 
   and start_time between '{{s}}' and '{{e}}' 
   and query_text != '' -- Added by JH, 2025-04-26
 group by query_text 
 order by exec_time desc 
 limit 10 ;

In [ ]:
st.title('지정된 기간내의 반복되는 쿼리 수행')

#fig_execution_time=px.bar(Repeated_Query,x='EXEC_TIME',y='QUERY_TEXT', orientation='h',title="Total Execution Time by Repeated Queries")
#fig_execution_time.update_traces(marker_color='LightSkyBlue')
#st.plotly_chart(fig_execution_time, use_container_width=True)

#Modifed by JH, 2025-04-26

#convert to pandas df
Total_Execution_Time_df = Repeated_Query_SQL.to_pandas()

max_len = 100
col_orig = 'QUERY_TEXT'           # 원본 텍스트 컬럼명
col_short = 'QUERY_TEXT_SHORT'    # 새로 만들 짧은 텍스트 컬럼명

# 지정된 길이로 텍스트 자르기
Total_Execution_Time_df[col_short] = Total_Execution_Time_df[col_orig].str.slice(0, max_len)
# 원본 길이가 max_len보다 긴 경우에만 끝에 '...' 추가
Total_Execution_Time_df.loc[Total_Execution_Time_df[col_orig].str.len() > max_len, col_short] += '...'

fig_Total_Execution_Time = px.bar(
    Total_Execution_Time_df,
    x='EXEC_TIME',
    y=col_short,          # Y축에는 새로 만든 짧은 텍스트 컬럼 사용
    orientation='h',
    title="Total Execution Time by Repeated Queries",
    hover_data={          # 마우스 호버 시 표시될 정보 설정
        'EXEC_TIME': ':.3f s',  # 실행 시간 (소수점 3자리 초 단위 포맷으로 수정)
        col_orig: True,         # 원본 전체 QUERY_TEXT 표시
        col_short: False,       # Y축 레이블과 동일한 짧은 텍스트는 호버에서 숨김 (선택 사항)
        'NUMBER_OF_QUERY': ':,d' # <<<--- 추가: 쿼리 실행 횟수 (정수형, 콤마 구분 포맷)
    },
    labels={               # 축 및 호버 레이블 이름 설정 (선택 사항)
        col_short: "Query Text (Truncated)",
        'EXEC_TIME': "Total Execution Time (s)",
        'NUMBER_OF_QUERY': "Number of Executions"
    }
)
fig_Total_Execution_Time.update_traces(marker_color='LightSkyBlue')

st.write(fig_Total_Execution_Time)


In [ ]:
--QUERIES THAT SPILL TO DISK
select top 25
       --hash(query_text) as query_hash
       query_text
      --,count(query_id) as query_count
      ,avg(total_elapsed_time) as avg_total_elapsed_time
      ,sum(bytes_spilled_to_local_storage) / 1024 / 1024 / 1024 as sum_gb_spilled_to_disk
      ,sum(bytes_spilled_to_remote_storage) / 1024 / 1024 / 1024 as sum_gb_spilled_to_blob
      ,sum(bytes_spilled_to_local_storage+bytes_spilled_to_remote_storage) as total_bytes
  from snowflake.account_usage.query_history
 where start_time between '{{s}}' and '{{e}}' 
   and query_text != ''
 group by query_text
--group by query_hash, query_text
--having query_count > 5
having total_bytes > 0
 order by total_bytes desc;

In [ ]:
st.title('지정된 기간 동안 Disk로 Spill된 쿼리 리스트')

#fig_execution_time=px.bar(Repeated_Query,x='EXEC_TIME',y='QUERY_TEXT', orientation='h',title="Total Execution Time by Repeated Queries")
#fig_execution_time.update_traces(marker_color='LightSkyBlue')
#st.plotly_chart(fig_execution_time, use_container_width=True)

#Modifed by JH, 2025-04-26

#convert to pandas df
SpillToDisk_df = Spill_to_Disk_SQL.to_pandas()

max_len = 100
col_orig = 'QUERY_TEXT'           # 원본 텍스트 컬럼명
col_short = 'QUERY_TEXT_SHORT'    # 새로 만들 짧은 텍스트 컬럼명

# 지정된 길이로 텍스트 자르기
SpillToDisk_df[col_short] = SpillToDisk_df[col_orig].str.slice(0, max_len)
# 원본 길이가 max_len보다 긴 경우에만 끝에 '...' 추가
SpillToDisk_df.loc[SpillToDisk_df[col_orig].str.len() > max_len, col_short] += '...'

fig_SpillToDisk = px.bar(
    SpillToDisk_df,
    x='TOTAL_BYTES',
    y=col_short,          # Y축에는 새로 만든 짧은 텍스트 컬럼 사용
    orientation='h',
    title="Spill to Disk Queries",
    hover_data={          # 마우스 호버 시 표시될 정보 설정
        'AVG_TOTAL_ELAPSED_TIME': ':.2f',  
        'TOTAL_BYTES': ':.d',  
        col_orig: True,         # 원본 전체 QUERY_TEXT 표시
        col_short: False     # Y축 레이블과 동일한 짧은 텍스트는 호버에서 숨김 (선택 사항)
    },
    labels={               # 축 및 호버 레이블 이름 설정 (선택 사항)
        col_short: "Query Text (Truncated)",
        'TOTAL_BYTES': "Spill to Disk",
        'AVG_TOTAL_ELAPSED_TIME': "Average Elapsed Time"
    }
)
fig_SpillToDisk.update_traces(marker_color='Yellow')

st.write(fig_SpillToDisk)

In [ ]:
--Top 10 Average Query Execution Time (By User)
select user_name, (avg(execution_time)) / 1000 as average_execution_time 
  from snowflake.account_usage.query_history 
 where execution_status = 'SUCCESS'            --Added by JH, 2025-04-26
   and start_time between '{{s}}' and '{{e}}'  --Added by JH, 2025-04-26
 group by 1 
 order by 2 desc limit 10 ;

In [ ]:
st.title('지정된 기간동안 사용자별 평균 쿼리 실행 시간 (초)')

query_execution_df = Query_Execution_By_User_SQL.to_pandas()
#st.write(query_execution_df)
fig_cquery_execution=px.bar(query_execution_df,x='USER_NAME',y='AVERAGE_EXECUTION_TIME', orientation='v',title="Average Execution Time per User")
fig_cquery_execution.update_traces(marker_color='MediumPurple')
st.plotly_chart(fig_cquery_execution,use_container_width=True)

In [ ]:
SELECT query_history.query_id,
       query_history.query_text,
       query_history.start_time,
       query_history.end_time,
       query_history.user_name,
       query_history.database_name,
       query_history.schema_name,
       query_history.warehouse_name,
       query_history.warehouse_size,
       metering_history.credits_used,
       execution_time/1000 as execution_time_s
  FROM snowflake.account_usage.query_history
  JOIN snowflake.account_usage.metering_history 
    ON query_history.start_time >= metering_history.start_time
   AND query_history.end_time <= metering_history.end_time
 WHERE query_history.start_time >= DATEADD(DAY, -7, CURRENT_TIMESTAMP())
 ORDER BY query_history.query_id;

In [ ]:
st.title('한 주간 시간대별 쿼리 사용량 #1 - Heatmap')

#import pandas as pd
#import streamlit as st
##import altair as alt

# Get data
df = Query_Heatmap_1W_SQL.to_pandas()

# Create date filter slider
st.subheader("Select time duration")

col = st.columns(3)

with col[0]:
    days = st.slider('Select number of days to analyze', 
                     min_value=1, 
                     max_value=7, 
                     value=7, 
                     step=1)
with col[1]:
    var = st.selectbox("Select a variable", ['WAREHOUSE_NAME', 'USER_NAME', 'WAREHOUSE_SIZE'])
with col[2]:
    metric = st.selectbox("Select a metric", ["COUNT", "TOTAL_CREDITS_USED"])

# Filter data according to day duration
df['START_TIME'] = pd.to_datetime(df['START_TIME'])
latest_date = df['START_TIME'].max()
cutoff_date = latest_date - pd.Timedelta(days=days)
filtered_df = df[df['START_TIME'] > cutoff_date].copy()
    
# Prepare data for heatmap
filtered_df['HOUR_OF_DAY'] = filtered_df['START_TIME'].dt.hour
filtered_df['HOUR_DISPLAY'] = filtered_df['HOUR_OF_DAY'].apply(lambda x: f"{x:02d}:00")
    
# Calculate frequency count and sum of credits by hour and query
agg_df = (filtered_df.groupby(['QUERY_ID', 'HOUR_DISPLAY', var])
          .agg(
              COUNT=('QUERY_ID', 'size'),
              TOTAL_CREDITS_USED=('CREDITS_USED', 'sum')
          )
          .reset_index()
)

st.warning(f"Analyzing {var} data for the last {days} days!")

## Heatmap
heatmap = alt.Chart(agg_df).mark_rect(stroke='black', strokeWidth=1).encode(
    x='HOUR_DISPLAY:O',
    y=alt.Y(f'{var}:N', 
            title='',
            axis=alt.Axis(
                labels=True,
                labelLimit=250,
                tickMinStep=1,
                labelOverlap=False,
                labelPadding=10
            )),
    color=f'{metric}:Q',
    tooltip=['HOUR_DISPLAY', var, metric]
).properties(
    title=f'Query Activity Heatmap by Hour and {var}'
)

st.altair_chart(heatmap, use_container_width=True)

In [ ]:
st.title('한 주간 시간대별 쿼리 사용량 #2 Stacked bar chart')

#import pandas as pd
#import streamlit as st
##import altair as alt

# Get data
df = Query_Heatmap_1W_SQL.to_pandas()

# Create date filter slider
st.subheader("Select time duration")

col = st.columns(3)

with col[0]:
    days = st.slider('Select number of days to analyze', 
                     min_value=1, 
                     max_value=7, 
                     value=7, 
                     step=1,
                     key='days_slider_2')
with col[1]:
    var = st.selectbox("Select a variable", ['WAREHOUSE_NAME', 'USER_NAME', 'WAREHOUSE_SIZE'],
                       key='variable_selectbox_2')  # 고유 키 추가
with col[2]:
    metric = st.selectbox("Select a metric", ["COUNT", "TOTAL_CREDITS_USED"],
                         key='metric_selectbox_2')  # 고유 키 추가

# Filter data according to day duration
df['START_TIME'] = pd.to_datetime(df['START_TIME'])
latest_date = df['START_TIME'].max()
cutoff_date = latest_date - pd.Timedelta(days=days)
filtered_df = df[df['START_TIME'] > cutoff_date].copy()
    
# Prepare data for heatmap
filtered_df['HOUR_OF_DAY'] = filtered_df['START_TIME'].dt.hour
filtered_df['HOUR_DISPLAY'] = filtered_df['HOUR_OF_DAY'].apply(lambda x: f"{x:02d}:00")
    
# Calculate frequency count and sum of credits by hour and query
agg_df = (filtered_df.groupby(['QUERY_ID', 'HOUR_DISPLAY', var])
          .agg(
              COUNT=('QUERY_ID', 'size'),
              TOTAL_CREDITS_USED=('CREDITS_USED', 'sum')
          )
          .reset_index()
)

st.warning(f"Analyzing {var} data for the last {days} days!")

## Stacked bar chart with time series
bar_time = alt.Chart(agg_df).mark_bar().encode(
    x='HOUR_DISPLAY:O',
    y=f'{metric}:Q',
    color=alt.Color(f'{var}:N', legend=alt.Legend(orient='bottom')),
    tooltip=['HOUR_DISPLAY', var, metric]
).properties(
    title=f'Query Activity by Hour and {var}',
    height=400
)

st.altair_chart(bar_time, use_container_width=True)


In [ ]:
st.title('한 주간 시간대별 쿼리 사용량 #3 - Bubble plot')

#import pandas as pd
#import streamlit as st
##import altair as alt

# Get data
df = Query_Heatmap_1W_SQL.to_pandas()

# Create date filter slider
st.subheader("Select time duration")

col = st.columns(3)

with col[0]:
    days = st.slider('Select number of days to analyze', 
                     min_value=1, 
                     max_value=7, 
                     value=7, 
                     step=1,
                     key='days_slider_3')
with col[1]:
    var = st.selectbox("Select a variable", ['WAREHOUSE_NAME', 'USER_NAME', 'WAREHOUSE_SIZE'],
                       key='variable_selectbox_3')
with col[2]:
    metric = st.selectbox("Select a metric", ["COUNT", "TOTAL_CREDITS_USED"],
                         key='metric_selectbox_3')

# Filter data according to day duration
df['START_TIME'] = pd.to_datetime(df['START_TIME'])
latest_date = df['START_TIME'].max()
cutoff_date = latest_date - pd.Timedelta(days=days)
filtered_df = df[df['START_TIME'] > cutoff_date].copy()
    
# Prepare data for heatmap
filtered_df['HOUR_OF_DAY'] = filtered_df['START_TIME'].dt.hour
filtered_df['HOUR_DISPLAY'] = filtered_df['HOUR_OF_DAY'].apply(lambda x: f"{x:02d}:00")
    
# Calculate frequency count and sum of credits by hour and query
agg_df = (filtered_df.groupby(['QUERY_ID', 'HOUR_DISPLAY', var])
          .agg(
              COUNT=('QUERY_ID', 'size'),
              TOTAL_CREDITS_USED=('CREDITS_USED', 'sum')
          )
          .reset_index()
)

st.warning(f"Analyzing {var} data for the last {days} days!")

## Bubble plot with size representing the metric
bubble = alt.Chart(agg_df).mark_circle().encode(
    x='HOUR_DISPLAY:O',
    y=alt.Y(f'{var}:N', title=''),
    size=alt.Size(f'{metric}:Q', legend=alt.Legend(title='Query Count')),
    color=alt.Color(f'{var}:N', legend=None),
    tooltip=['HOUR_DISPLAY', var, metric]
).properties(
    title=f'Query Distribution by Hour and {var}',
    height=550
)

st.altair_chart(bubble, use_container_width=True)

# 클라우드 서비스 영역

In [ ]:
--GS Utilization by Query Type (Top 10)
select query_type, 
       sum(credits_used_cloud_services) cs_credits, 
       count(1) num_queries 
  from snowflake.account_usage.query_history 
 where true 
   and start_time between '{{s}}' and '{{e}}'  --Added by JH, 2025-04-26
 group by 1 
 order by 2 desc 
 limit 10 ;

In [ ]:
st.title('지정된 기간동안 쿼리 유형별 클라우드서비스 총 사용 크레딧량')

gs_utilization_df = GS_Util_By_Query_Type_SQL.to_pandas()
#fig_gs_utilization=px.bar(gs_utilization_df,x='QUERY_TYPE',y='CS_CREDITS', orientation='v',title="GS Utilization by Query Type (Top 10)")

#Added by JH, 2025-04-27
fig_gs_utilization = px.bar(
    gs_utilization_df,
    x='QUERY_TYPE',
    y='CS_CREDITS',
    orientation='v', # Vertical orientation
    title="GS Utilization by Query Type (Top 10)",
    hover_data={ # Define data to show on hover
        'QUERY_TYPE': False, # Hide QUERY_TYPE in hover (already on x-axis)
        'CS_CREDITS': ':.2f credits', # Format CS_CREDITS (e.g., 2 decimal places)
        'NUM_QUERIES': ':,d' # Add NUM_QUERIES, format as integer with commas
    },
    labels={ # Customize labels shown in hover and on axes
        'CS_CREDITS': 'Cloud Service Credits',
        'NUM_QUERIES': 'Number of Queries'
    }
)
fig_gs_utilization.update_traces(marker_color='green')

In [ ]:
--Top 10 Cloud Services by Warehouse   
select warehouse_name, 
       sum(credits_used_cloud_services) CREDITS_USED_CLOUD_SERVICES 
  from snowflake.account_usage.warehouse_metering_history 
 where true 
   and start_time between '{{s}}' and '{{e}}'  --Added by JH, 2025-04-26
 group by 1 
 order by 2 desc limit 10;

In [ ]:
st.title('지정된 기간동안 웨어하우스별 클라우드서비스의 총 사용 크레딧양')

compute_gs_by_warehouse_df = Cloud_Service_By_Warehouse_SQL.to_pandas()
#st.write(compute_gs_by_warehouse_df)
fig_compute_gs_by_warehouse=px.bar(compute_gs_by_warehouse_df,x='WAREHOUSE_NAME',y='CREDITS_USED_CLOUD_SERVICES', orientation='v',title="Compute and Cloud Services by Warehouse", barmode="group")
fig_compute_gs_by_warehouse.update_traces(marker_color='purple')

# 스토리지

In [ ]:
--Data Storage used Overtime 
select date_trunc(month, usage_date) as usage_month, 
       avg(storage_bytes + stage_bytes + failsafe_bytes) / power(1024, 4) as billable_tb, 
       avg(storage_bytes) / power(1024, 4) as Storage_TB, 
       avg(stage_bytes) / power(1024, 4) as Stage_TB, 
       avg(failsafe_bytes) / power(1024, 4) as Failsafe_TB 
  from snowflake.account_usage.storage_usage 
 group by 1 
 order by 1 ;

In [ ]:
st.title('월별 스토리지 사용량 추이')

# storage_overtime_df = Storage_Overtime_SQL.to_pandas()

# fig_storage_overtime=px.bar(storage_overtime_df,x='USAGE_MONTH',y='BILLABLE_TB', orientation='v',title="Data Storage used Overtime", barmode="group")
# st.plotly_chart(fig_storage_overtime, use_container_width=True)

# st.info('The above chart is static and non modified by the date range filter', icon="ℹ️")

# Modified by JH, 2025-04-27
storage_overtime_df = Storage_Overtime_SQL.to_pandas()

# --- 데이터 변환 (Wide to Long) ---
storage_df_melted = pd.melt(
    storage_overtime_df,
    id_vars=['USAGE_MONTH'], # 기준이 되는 컬럼
    value_vars=['STORAGE_TB', 'STAGE_TB', 'FAILSAFE_TB'], # 누적할 값 컬럼들
    var_name='STORAGE_TYPE', # 스토리지 유형을 나타낼 새 컬럼 이름
    value_name='SIZE_TB'      # 스토리지 크기 값을 저장할 새 컬럼 이름
)

# (선택 사항) 스토리지 유형 순서 지정 (예: Storage -> Stage -> Failsafe)
storage_type_order = ['STORAGE_TB', 'STAGE_TB', 'FAILSAFE_TB']
storage_df_melted['STORAGE_TYPE'] = pd.Categorical(
    storage_df_melted['STORAGE_TYPE'], categories=storage_type_order, ordered=True
)
storage_df_melted = storage_df_melted.sort_values(by=['USAGE_MONTH', 'STORAGE_TYPE'])


# --- 누적 막대 차트 생성 ---
fig_stacked_storage = px.bar(
    storage_df_melted,
    x='USAGE_MONTH',    # x축: 월 (또는 다른 기준 컬럼)
    y='SIZE_TB',          # y축: 스토리지 크기
    color='STORAGE_TYPE', # 이 컬럼 기준으로 막대를 누적하고 색상 구분
    title="Data Storage used Overtime", # 차트 제목
    orientation='v',      # 세로 막대 차트
    hover_data={          # 마우스 호버 시 표시될 정보 설정
        'USAGE_MONTH': True,
        'STORAGE_TYPE': True,
        'SIZE_TB': ':.2f TB' # 스토리지 크기 (소수점 2자리 TB 단위 포맷)
    },
    labels={              # 축 및 범례/호버 레이블 이름 설정
        'USAGE_MONTH': 'Month',
        'SIZE_TB': 'Storage Size (TB)',
        'STORAGE_TYPE': 'Storage Type'
    },
    # color_discrete_map={ # 특정 색상 지정 (선택 사항)
    #     'STORAGE_TB': 'blue',
    #     'STAGE_TB': 'orange',
    #     'FAILSAFE_TB': 'red'
    # }
)

st.plotly_chart(fig_stacked_storage) # Streamlit 환경에서 사용

st.info('The above chart is static and non modified by the date range filter', icon="ℹ️")

In [ ]:
--Rows Loaded Overtime (COPY INTO) 
select to_timestamp(date_trunc(day,last_load_time)) as usage_date, 
       sum(row_count) as total_rows 
  from snowflake.account_usage.load_history 
 where usage_date between '{{s}}' and '{{e}}' 
 group by 1 
 order by usage_date desc ;

In [ ]:
st.title('지정된 기간동안 일별 데이터 적재건수 추이')

rows_loaded_df = Rows_Loaded_SQL.to_pandas()

fig_rows_loaded=px.bar(rows_loaded_df,x='USAGE_DATE',y='TOTAL_ROWS', orientation='v',title="Rows Loaded Overtime (Copy Into)")
st.plotly_chart(fig_rows_loaded, use_container_width=True)

# 사용자

In [ ]:
--Logins by User 
select user_name, 
       sum(iff(is_success = 'NO', 1, 0)) as Failed, 
       count(*) as Success, 
       sum(iff(is_success = 'NO', 1, 0)) / nullif(count(*), 0) as login_failure_rate 
  from snowflake.account_usage.login_history 
 where true
   and event_timestamp between '{{s}}' and '{{e}}' --Added by JH, 2025-04-27
 group by 1 
 order by 4 desc;


In [ ]:
st.title('지정된 기간동안 사용자별 로그인 성공/실패 횟수')

logins_df_wide = Login_User_SQL.to_pandas()

#Modified by JH, 2025-04-27
logins_df_melted = pd.melt(
    logins_df_wide,
    id_vars=['USER_NAME'],              # 기준이 되는 컬럼
    value_vars=['SUCCESS', 'FAILED'],   # 누적할 값 컬럼들
    var_name='LOGIN_STATUS',            # 로그인 상태를 나타낼 새 컬럼 이름
    value_name='COUNT'                  # 로그인 횟수 값을 저장할 새 컬럼 이름
)

# (선택 사항) 로그인 상태 순서 지정 (예: Successful -> Failed)
status_order = ['SUCCESS', 'FAILED']
logins_df_melted['LOGIN_STATUS'] = pd.Categorical(
    logins_df_melted['LOGIN_STATUS'], categories=status_order, ordered=True
)
logins_df_melted = logins_df_melted.sort_values(by=['USER_NAME', 'LOGIN_STATUS'])


# --- 누적 막대 차트 생성 ---
fig_logins_stacked = px.bar(
    logins_df_melted,
    x='USER_NAME',        # x축: 사용자 이름
    y='COUNT',            # y축: 로그인 횟수
    color='LOGIN_STATUS', # 이 컬럼 기준으로 막대를 누적하고 색상 구분
    title="Sucessful & Failed login by Users", # 차트 제목
    orientation='v',      # 세로 막대 차트
    hover_data={          # 마우스 호버 시 표시될 정보 설정
        'USER_NAME': True,
        'LOGIN_STATUS': True,
        'COUNT': ':,d'    # 로그인 횟수 (정수형, 콤마 구분 포맷)
    },
    labels={              # 축 및 범례/호버 레이블 이름 설정
        'USER_NAME': '사용자 이름',
        'COUNT': '로그인 횟수',
        'LOGIN_STATUS': '로그인 상태'
    },
    color_discrete_map={ # 색상 지정 (선택 사항)
        'SUCCESS': 'lightgreen',
        'FAILED': 'red'
    }
    # barmode='stack' # 'color' 사용 시 기본값이 'stack'이므로 명시적으로 지정할 필요 없음
)

# (선택 사항) x축 순서 정렬 (예: 총 로그인 횟수 기준 내림차순)
# total_logins = logins_df_wide.set_index('USER_NAME')[['SUCCESSFUL', 'FAILED']].sum(axis=1).sort_values(ascending=False)
# fig_logins_stacked.update_layout(xaxis={'categoryorder':'array', 'categoryarray': total_logins.index})


# --- Streamlit에 차트 표시 ---
st.plotly_chart(fig_logins_stacked, use_container_width=True) # Streamlit 환경에서 사용

#fig_logins_stacked.show() # 로컬 환경 등에서 확인시

In [ ]:
--Logins by Client 
select reported_client_type as Client, 
       user_name, 
       sum(iff(is_success = 'NO', 1, 0)) as Failed, 
       count(*) as Success, 
       sum(iff(is_success = 'NO', 1, 0)) / nullif(count(*), 0) as login_failure_rate 
  from snowflake.account_usage.login_history 
 where true
   and event_timestamp between '{{s}}' and '{{e}}' --Added by JH, 2025-04-27 
 group by 1, 2 
 order by 5 desc ;


In [ ]:
st.title('지정된 기간동안 클라이언트 유형별, 사용자별 로그인 성공/실패 횟수')

# logins_client_df = Login_Client_SQL.to_pandas()

# fig_logins_client=px.bar(logins_client_df,x='CLIENT',y='SUCCESS', orientation='v',title="Logins by Client")
# fig_logins_client.update_traces(marker_color='purple')

# Modified by JH, 2025-04-27
logins_client_df_wide = Login_Client_SQL.to_pandas()

logins_client_df_melted = pd.melt(
    logins_client_df_wide,
    id_vars=['CLIENT', 'USER_NAME'],        # 기준이 되는 컬럼들
    value_vars=['SUCCESS', 'FAILED'],    # 그룹화/색상 구분할 값 컬럼들
    var_name='LOGIN_STATUS',                # 로그인 상태를 나타낼 새 컬럼 이름
    value_name='COUNT'                      # 로그인 횟수 값을 저장할 새 컬럼 이름
)

# (선택 사항) 로그인 상태 순서 지정 (예: Successful -> Failed)
status_order = ['SUCCESS', 'FAILED']
logins_client_df_melted['LOGIN_STATUS'] = pd.Categorical(
    logins_client_df_melted['LOGIN_STATUS'], categories=status_order, ordered=True
)
logins_client_df_melted = logins_client_df_melted.sort_values(by=['CLIENT', 'USER_NAME', 'LOGIN_STATUS'])

fig_logins_client_grouped = px.bar(
    logins_client_df_melted,
    x='USER_NAME',        # 각 패싯 내 x축: 사용자 이름
    y='COUNT',            # y축: 로그인 횟수
    color='LOGIN_STATUS', # 막대 색상 구분: 로그인 상태 (성공/실패)
    facet_col='CLIENT',   # 패싯 열: 클라이언트 유형별로 하위 그래프 생성
    facet_col_wrap=3,     # 한 줄에 표시할 최대 패싯 수 (조정 가능)
    title="Logins by Client", # 차트 제목
    hover_data={          # 마우스 호버 시 표시될 정보 설정
        'CLIENT': True,
        'USER_NAME': True,
        'LOGIN_STATUS': True,
        'COUNT': ':,d'    # 로그인 횟수 (정수형, 콤마 구분 포맷)
    },
    labels={              # 축 및 범례/호버 레이블 이름 설정
        'USER_NAME': 'Username',
        'COUNT': 'Number of logins',
        'LOGIN_STATUS': 'Login status',
        'CLIENT': 'Client type'
    },
    color_discrete_map={ # 색상 지정 (선택 사항)
        'SUCCESS': 'mediumseagreen', # 성공 색상 변경
        'FAILED': 'tomato'            # 실패 색상 변경
    },
    height=500 # 차트 높이 조절 (패싯 수에 따라 조정)
)

# 패싯 제목 스타일 조정 (선택 사항)
fig_logins_client_grouped.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# x축 레이블 각도 조정 (사용자 이름이 많을 경우)
fig_logins_client_grouped.update_xaxes(tickangle=45)

st.plotly_chart(fig_logins_client_grouped, use_container_width=True) # Streamlit 환경에서 사용



In [ ]:
--Never Logged In Users
SHOW USERS;
SELECT * FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
WHERE "last_success_login" IS NULL
AND DATEDIFF('Day',"created_on",CURRENT_DATE) > 30;

In [ ]:
st.title('생성 후 30일 이내 로그인한 이력이 없는 사용자 ')

df_users = Never_Login_Since_Created_SQL.to_pandas()

if not df_users.empty:
    df_users['created_on'] = pd.to_datetime(df_users['created_on'])

    st.subheader("사용자 생성 타임라인")

    # --- Plotly Express로 산점도 생성 ---
    fig_timeline = px.scatter(
        df_users, 
        x='created_on',     
        y='name',           
        title='Inactive User Creation Timeline',
        labels={            
            'created_on': 'Creation Date/Time',
            'name': 'User Name'
        },
        hover_name='name',  
        hover_data={       
            'created_on': '|%Y-%m-%d %H:%M:%S', 
            'name': False 
        }
    )

    # --- 차트 레이아웃 커스터마이징 ---
    fig_timeline.update_layout(
        xaxis_title='Creation Date/Time',
        yaxis_title='User Name',
        height=max(400, len(df_users['name'].unique()) * 20) # 사용자 수에 따라 높이 조절
        # Y축 사용자 이름을 알파벳 오름차순으로 정렬
        # yaxis_categoryorder='category ascending' # 또는 category descending
    )
    # Y축 레이블 정렬 (알파벳 순) - 필요시 주석 해제
    fig_timeline.update_yaxes(categoryorder='category ascending')

    # 점(마커) 스타일 변경 (선택 사항)
    fig_timeline.update_traces(marker=dict(size=8, symbol='circle'))

    # --- Streamlit에 차트 표시 ---
    st.plotly_chart(fig_timeline, use_container_width=True)

else:
    st.warning("조건에 맞는 사용자가 없습니다.")

In [ ]:
--Stale Users
SHOW USERS;
SELECT * FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
WHERE DATEDIFF('Day',"last_success_login",CURRENT_DATE) > 30;

In [ ]:
st.title('지난 30일 이내 새로 로그인하지 않은 사용자')

df_users = Infrequent_User_SQL.to_pandas()

if not df_users.empty:
    df_users['last_success_login'] = pd.to_datetime(df_users['last_success_login'])

    st.subheader("사용자 생성 타임라인")

    # --- Plotly Express로 산점도 생성 ---
    fig_timeline = px.scatter(
        df_users, 
        x='last_success_login',     
        y='name',           
        title='Infrequent User',
        labels={            
            'last_success_login': 'last success login Date/Time',
            'name': 'User Name'
        },
        hover_name='name',  
        hover_data={       
            'last_success_login': '|%Y-%m-%d %H:%M:%S', 
            'name': False 
        }
    )

    # --- 차트 레이아웃 커스터마이징 ---
    fig_timeline.update_layout(
        xaxis_title='last success login Date/Time',
        yaxis_title='User Name',
        height=max(400, len(df_users['name'].unique()) * 20) # 사용자 수에 따라 높이 조절
        # Y축 사용자 이름을 알파벳 오름차순으로 정렬
        # yaxis_categoryorder='category ascending' # 또는 category descending
    )
    # Y축 레이블 정렬 (알파벳 순) - 필요시 주석 해제
    fig_timeline.update_yaxes(categoryorder='category ascending')

    # 점(마커) 스타일 변경 (선택 사항)
    fig_timeline.update_traces(marker=dict(size=8, symbol='circle'))

    # --- Streamlit에 차트 표시 ---
    st.plotly_chart(fig_timeline, use_container_width=True)

else:
    st.warning("조건에 맞는 사용자가 없습니다.")

# 테이블 분석

In [ ]:

-- Identify high churn tables or short lived tables
SELECT
        t.table_catalog||'.'||t.table_schema||'.'||t.table_name as fq_table_name
       ,t.table_catalog as database
       ,t.table_schema as schema
       ,t.table_name as table_name
       ,t.active_bytes/power(1024,3) as active_size_gb
       ,t.time_travel_bytes/power(1024,3) as time_travel_gb
       ,t.failsafe_bytes/power(1024,3) as failsafe_gb
       ,t.retained_for_clone_bytes/power(1024,3) as clone_retain_gb
       ,active_size_gb+time_travel_gb+failsafe_gb+clone_retain_gb as total_size_gb
       ,(t.time_travel_bytes + t.failsafe_bytes + t.retained_for_clone_bytes)/power(1024,3) as non_active_size_gb
       ,div0(non_active_size_gb,active_size_gb)*100 as churn_pct
       ,t.deleted
       ,timediff('hour',t.table_created,t.table_dropped) as table_life_duration_hours
       ,t1.is_transient
       ,t1.table_type
       ,t1.retention_time
       ,t1.auto_clustering_on
       ,t1.clustering_key
       ,t1.last_altered
       ,t1.last_ddl
   FROM snowflake.account_usage.table_storage_metrics t
        JOIN snowflake.account_usage.tables t1
          ON t.id=t1.table_id
  WHERE 1=1
        AND t1.table_catalog not in ('SNOWFLAKE') -- use this to filter on specific databases
        AND 
            (
             churn_pct>=40
             OR
             table_life_duration_hours<=24  -- short lived tables
            )
  ORDER BY total_size_gb desc ;

In [ ]:
# Modifed by JH, 2025-05-2

# Streamlit 앱 제목 설정
st.title('짧은 생명 주기의 테이블 리스트')

# 가정: HighChurn_ShortLived_SQL 변수에 쿼리 결과가 있고, 이를 Pandas DataFrame으로 변환합니다.
HighChurn_ShortLived_df = HighChurn_ShortLived_SQL.to_pandas()

# 1. 데이터베이스별 테이블 수 시각화 (막대 그래프)
st.subheader("데이터베이스별 테이블 수")
fig_db = px.bar(HighChurn_ShortLived_df, x='DATABASE', color='DATABASE',
             title='데이터베이스별 테이블 수',
             labels={'DATABASE': '데이터베이스', 'count': '테이블 수'})
fig_db.update_layout(xaxis_tickangle=-45)
st.plotly_chart(fig_db)

# 2. 스키마별 테이블 수 시각화 (막대 그래프)
st.subheader("스키마별 테이블 수")
fig_schema = px.bar(HighChurn_ShortLived_df, x='SCHEMA', color='SCHEMA',
                 title='스키마별 테이블 수',
                 labels={'SCHEMA': '스키마', 'count': '테이블 수'})
fig_schema.update_layout(xaxis_tickangle=-45)
st.plotly_chart(fig_schema)

# 3. 테이블별 총 크기 시각화 (수평 막대 그래프 - 테이블 이름 확인 용이)
st.subheader("테이블별 총 크기 (GB)")
df_sorted = HighChurn_ShortLived_df.sort_values(by='TOTAL_SIZE_GB', ascending=False)
fig_size = px.bar(df_sorted, y='DATABASE', x='TOTAL_SIZE_GB', color='TABLE_NAME',
                 title='테이블별 총 크기 (GB)',
                 labels={'DATABASE': '데이터베이스', 'TOTAL_SIZE_GB': '총 크기 (GB)', 'TABLE_NAME': '테이블 이름'},
                 orientation='h')
fig_size.update_layout(yaxis={'categoryorder':'array', 'categoryarray': df_sorted['DATABASE'].unique()})
st.plotly_chart(fig_size)

# 추가적으로 테이블 이름과 크기를 함께 보여주는 테이블 형태 (plotly.graph_objects 사용)
st.subheader("테이블 이름 및 총 크기")
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(HighChurn_ShortLived_df[['DATABASE', 'SCHEMA', 'TABLE_NAME', 'DELETED','IS_TRANSIENT','LAST_ALTERED','LAST_DDL']].columns),
                align='left'),
    cells=dict(values=[HighChurn_ShortLived_df['DATABASE'], HighChurn_ShortLived_df['SCHEMA'], HighChurn_ShortLived_df['TABLE_NAME'], HighChurn_ShortLived_df['DELETED'],HighChurn_ShortLived_df['IS_TRANSIENT'],HighChurn_ShortLived_df['LAST_ALTERED'], HighChurn_ShortLived_df['LAST_DDL']],
               align='left'))
])
fig_table.update_layout(title='테이블 이름 및 총 크기')
st.plotly_chart(fig_table)

In [ ]:
-- Unused tables
-- Identify Table sizes and Last DDL/DML Timestamps
SELECT TABLE_CATALOG || '.' || TABLE_SCHEMA || '.' || TABLE_NAME AS TABLE_PATH
      ,TABLE_CATALOG AS DATABASE
      ,TABLE_SCHEMA AS SCHEMA
      ,TABLE_NAME
      ,BYTES
      ,TO_NUMBER(BYTES / POWER(1024,3),10,2) AS GB
      ,LAST_ALTERED AS LAST_USE
      ,DATEDIFF('Day',LAST_USE,CURRENT_DATE) AS DAYS_SINCE_LAST_USE
  FROM INFORMATION_SCHEMA.TABLES
 WHERE DAYS_SINCE_LAST_USE > 7 --Use your Days Threshold
 ORDER BY DATABASE, SCHEMA, TABLE_NAME, DAYS_SINCE_LAST_USE DESC;

In [ ]:
st.title('7일 이상 사용되지 않은 테이블 리스트')

UnusedTable_df = Unused_Table_1W_SQL.to_pandas()

# 선버스트 차트에서 각 테이블의 크기를 동일하게 표현하기 위해 카운트 컬럼 추가
# UnusedTable_df['COUNT'] = 1

# # --- 선버스트 차트(Sunburst Chart) 생성 ---
# fig_db_schema_table = px.sunburst(
#     UnusedTable_df,
#     path=['DATABASE', 'SCHEMA', 'TABLE_NAME'], # 계층 구조 정의: 데이터베이스 -> 스키마 -> 테이블
#     values='COUNT',                            # 각 조각(테이블)의 크기 (여기서는 동일하게 1로 설정)
#     title='Unused Table List',                 # 차트 제목
#     # color='DATABASE_NAME',                   # 최상위 레벨(데이터베이스) 기준으로 색상 구분 (선택 사항)
#     # maxdepth=2                               # 표시할 최대 깊이 설정 (예: 스키마까지만 보려면 2) (선택 사항)
# )

# # 차트 레이아웃 업데이트 (선택 사항)
# fig_db_schema_table.update_layout(
#     margin=dict(t=50, l=25, r=25, b=25) # 여백 조정
# )

# # 호버 정보 업데이트 (선택 사항)
# fig_db_schema_table.update_traces(
#     hovertemplate='<b>%{label}</b><br>경로: %{id}<extra></extra>' # 호버 시 표시될 텍스트 형식
# )

# # --- Streamlit에 차트 표시 ---
# st.plotly_chart(fig_db_schema_table, use_container_width=True) # Streamlit 환경에서 사용

df_sorted = UnusedTable_df.sort_values(by='GB', ascending=False)
fig_size = px.bar(df_sorted, y='DATABASE', x='GB', color='TABLE_PATH',
                 title='테이블별 총 크기 (GB)',
                 labels={'DATABASE': '데이터베이스', 'GB': '총 크기 (GB)', 'TABLE_PATH': '테이블 이름'},
                 orientation='h')
fig_size.update_layout(yaxis={'categoryorder':'array', 'categoryarray': df_sorted['DATABASE'].unique()})
st.plotly_chart(fig_size)

# 추가적으로 테이블 이름과 크기를 함께 보여주는 테이블 형태 (plotly.graph_objects 사용)
fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(UnusedTable_df[['DATABASE', 'SCHEMA', 'TABLE_NAME','GB']].columns),
                align='left'),
    cells=dict(values=[UnusedTable_df['DATABASE'], UnusedTable_df['SCHEMA'], UnusedTable_df['TABLE_NAME'], UnusedTable_df['GB']],
               align='left'))
])
fig_table.update_layout(title='테이블 이름 및 총 크기')
st.plotly_chart(fig_table)


In [ ]:
-- Tables used in any query in the last 7 days
WITH access_history as
(   
SELECT  
       distinct split(base.value:objectName, '.')[0]::string as DATABASE
       ,split(base.value:objectName, '.')[1]::string as SCHEMA
       ,split(base.value:objectName, '.')[2]::string as TABLE_NAME
  FROM snowflake.account_usage.access_history 
       ,lateral flatten (base_objects_accessed) base
 where query_start_time between current_date()-7 and current_date()
)
SELECT tbl.table_catalog||'.'||tbl.table_schema||'.'||tbl.table_name as FQ_table_name,
       ah.database,
       ah.schema,
       ah.table_name,
       TO_NUMBER(tbl.bytes / POWER(1024,3),10,2) AS GB
  FROM snowflake.account_usage.tables tbl
  LEFT JOIN access_history ah
    ON tbl.table_name=ah.table_name
   AND tbl.table_schema=ah.schema
   AND tbl.table_catalog=ah.database
 WHERE ah.table_name is not NULL
   AND tbl.deleted is null 
;


In [ ]:
# st.title('지난 7일간 쿼리에서 더 이상 사용되지 않은 테이블 리스트')

# TablesNotUsedInQuery_df = Tables_Not_Used_In_Query_SQL.to_pandas()

# # 선버스트 차트에서 각 테이블의 크기를 동일하게 표현하기 위해 카운트 컬럼 추가
# TablesNotUsedInQuery_df['COUNT'] = 1

# # --- 선버스트 차트(Sunburst Chart) 생성 ---
# fig_db_schema_table = px.sunburst(
#     TablesNotUsedInQuery_df,
#     path=['DATABASE', 'SCHEMA', 'TABLE_NAME'],               # 계층 구조 정의: 데이터베이스 -> 스키마 -> 테이블
#     values='COUNT',                                          # 각 조각(테이블)의 크기 (여기서는 동일하게 1로 설정)
#     title='Tables not used in any query in the last 7 days', # 차트 제목
#     # color='DATABASE_NAME',                                 # 최상위 레벨(데이터베이스) 기준으로 색상 구분 (선택 사항)
#     # maxdepth=2                                             # 표시할 최대 깊이 설정 (예: 스키마까지만 보려면 2) (선택 사항)
# )

# # 차트 레이아웃 업데이트 (선택 사항)
# fig_db_schema_table.update_layout(
#     margin=dict(t=50, l=25, r=25, b=25) # 여백 조정
# )

# # 호버 정보 업데이트 (선택 사항)
# fig_db_schema_table.update_traces(
#     hovertemplate='<b>%{label}</b><br>경로: %{id}<extra></extra>' # 호버 시 표시될 텍스트 형식
# )


# # --- Streamlit에 차트 표시 ---
# st.plotly_chart(fig_db_schema_table, use_container_width=True) # Streamlit 환경에서 사용

# modified by JH, 2025-05-02
st.title('지난 7일간 쿼리에서 사용된 테이블 리스트')

TablesUsedInQuery_df = Tables_Used_In_Query_SQL.to_pandas()

df_sorted = TablesUsedInQuery_df.sort_values(by='GB', ascending=False)
fig_size = px.bar(df_sorted, y='DATABASE', x='GB', color='TABLE_NAME',
                 title='테이블별 총 크기 (GB)',
                 labels={'DATABASE': '데이터베이스', 'GB': '총 크기 (GB)', 'TABLE_NAME': '테이블 이름'},
                 orientation='h')
fig_size.update_layout(yaxis={'categoryorder':'array', 'categoryarray': df_sorted['DATABASE'].unique()})
st.plotly_chart(fig_size)

fig_table = go.Figure(data=[go.Table(
    header=dict(values=list(df_sorted[['DATABASE', 'SCHEMA', 'TABLE_NAME','GB']].columns),
                align='left'),
    cells=dict(values=[df_sorted['DATABASE'], df_sorted['SCHEMA'], df_sorted['TABLE_NAME'], df_sorted['GB']],
               align='left'))
])
fig_table.update_layout(title='테이블 이름 및 총 크기')
st.plotly_chart(fig_table)